In [1]:
%%capture
!pip install geedim  # the needed dependencies
!pip install geojson
!pip install geemap
!pip install rasterio

In [2]:
import os.path
out_dir = os.path.expanduser('~/Downloads')
import zipfile
import os # we import hte necessary packages

# Import necessary modules
import geemap
import ee
import numpy as np
import rasterio # here we want to plot the data we received
import matplotlib.pyplot as plt
from geopy import distance
import copy
from geopy import distance
import copy

In [3]:
ee.Authenticate() # we authenticate
# and intialize geemap
ee.Initialize(project='ee-dataholderml3')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
from google.colab import drive # we mount the drive for later downloading of the files
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

# Define the region of interest (ROI) over Europe
roiComputed = ee.Geometry.Polygon(
    [[[9.5, 47.2],   # Southwestern corner
      [13.9, 47.2],  # Southeastern corner
      [13.9, 50.6],  # Northeastern corner
      [9.5, 50.6]]]  # Northwestern corner
)

In [6]:
import os
import numpy as np
import ee
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# Initialize the Earth Engine module
ee.Initialize()

# Define the region of interest (ROI) over Bavaria
roi = ee.Geometry.Polygon(
    [[[9.5, 47.2],   # Southwestern corner
      [13.9, 47.2],  # Southeastern corner
      [13.9, 50.6],  # Northeastern corner
      [9.5, 50.6]]]  # Northwestern corner
)

# Define the parent directory where the folders will be created
parent_directory = '/content/'

# Define the country (Germany in this case)
country = 'Germany'

# Create a folder for the country
folder_path = os.path.join(parent_directory, country)
os.makedirs(folder_path, exist_ok=True)

# Generate random locations
bounds = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', country)) \
    .geometry()

# Define the number of random points to generate
num_points_total = 1000000
num_points_per_iteration = 5000

# Initialize an empty DataFrame to store the coordinates
df_total = pd.DataFrame(columns=['Longitude', 'Latitude'])

# Initialize an empty list to store the DataFrames
df_list = []
i = 0
# Generate random points within the bounds of the geometry
for _ in range(num_points_total // num_points_per_iteration):
    i += 1
    print(i)
    random_points = ee.FeatureCollection.randomPoints(roi, num_points_per_iteration)

    # Get the information of the points
    list_of_dicts = random_points.getInfo()['features']

    # Initialize an empty list to store the coordinates
    coordinates = []

    # Iterate over the list of dictionaries
    for item in list_of_dicts:
        # Access the nested 'geometry' dictionary
        geometry_dict = item['geometry']
        # Access the 'coordinates' from the nested 'geometry' dictionary
        coords = geometry_dict['coordinates']
        # Append the coordinates to the list
        coordinates.append(coords)

    # Convert the 'coordinates' list to a DataFrame and append it to the list of DataFrames
    df = pd.DataFrame(coordinates, columns=['Longitude', 'Latitude'])
    df_list.append(df)

# Concatenate all DataFrames in the list
df_total = pd.concat(df_list, ignore_index=True)



1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [7]:
df_total_bavaria2 = df_total.copy(deep=True)


In [8]:
from shapely.geometry import Point, Polygon
polygon = Polygon([(9.5, 47.2), (13.9, 47.2), (13.9, 50.6), (9.5, 50.6)])  # Example polygon


In [9]:
df_total.to_csv('df_total.csv', index=False)


# Get the raster equivalent

In [10]:
import re
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import transform

In [11]:

import pandas as pd

# Load the dataframe
df_sample = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/dataframe_points_Bavaria.csv')



In [12]:
df_sample

,Longitude,Latitude
0,9.546802,47.866430
1,10.576114,50.424199
2,12.870875,47.267871
3,12.008238,47.817804
4,9.894532,48.561883
...,...,...
149995,12.726839,49.272750
149996,10.892782,49.599303
149997,13.307770,49.339694
149998,12.916161,48.165247


In [13]:
MODIS_NPP_Path_yearly = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP'
Elevation_Path_static = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/StaticValue/Elevation'
LST_Path_yearly = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/LST'
LAI_Path_yearly = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/LAI'
SoilEvaporation_Path_yearly = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/SoilEvaporation'
TotalEvaporation_Path_yearly = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/TotalEvapotranspiration'
LAI_Path_Season = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/LAI'
LST_Path_Season = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/LST'
SoilEvaporation_Path_Season = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/SoilEvaporation'
TotalEvaporation_Path_Season = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/TotalEvapotranspiration'


In [14]:
data_paths = [
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/StaticValue/Elevation',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/LST',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/LAI',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/SoilEvaporation',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/TotalEvapotranspiration',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/LAI',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/LST',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/SoilEvaporation',
    '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/TotalEvapotranspiration'
]


In [15]:

def build_kdtree_from_npy_file(file_path):
    # Load data from .npy file
    data = np.load(file_path)

    # Extract longitude and latitude columns
    longitude = data[:, 1]
    latitude = data[:, 2]

    # Build KD-tree
    tree = cKDTree(np.column_stack((longitude, latitude)))

    return tree

def find_closest_indices(tree, query_point, k=4):
    # Query the tree for k nearest neighbors
    distances, indices = tree.query(query_point, k=k)

    return indices

In [16]:

def generate_filename_dict(folder_path):
    filename_dict = {}

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Extract the ID number from the filename
        match = re.search(r'ID(\d+)N', filename)
        if match:
            id_number = int(match.group(1))

            # Store the filename in the dictionary with the ID number as the key
            filename_dict[id_number] = file_path

    return filename_dict

In [ ]:
file_dict = {}
crs_dict = {}

for path in data_paths:
  for root, dirs, files in os.walk(path):
        for dir in dirs:
            dir_path = os.path.join(root, path)
            for file in os.listdir(dir_path):
                if file.endswith('.npy') and dir in file:

                    subfolder = os.path.join(path,dir)
                                        # Get a list of all files in the subfolder
                    files = os.listdir(subfolder)

                    # Sort the list of files
                    files.sort()

                    # Get the first file
                    first_file = files[0]

                    # If you need the full path to the file
                    first_file_path = os.path.join(subfolder, first_file)
                    src_dataset = rasterio.open(first_file_path)
                    # Deep copy the source CRS
                    src_crs = copy.deepcopy(src_dataset.crs)
                    # Close the source dataset
                    src_dataset.close()
                        # Iterate over the files in the folder
                    transform_dict = {}
                    for filename in os.listdir(subfolder):
                        file_path = os.path.join(subfolder, filename)

                        # Extract the ID number from the filename
                        match = re.search(r'ID(\d+)N', filename)
                        if match:
                            id_number = int(match.group(1))

                            # Read the rasterio src.transform
                            with rasterio.open(file_path) as src:
                                src_transform_copy = copy.deepcopy(src.transform)

                            # Store the transform in the dictionary with the ID number as the key
                            transform_dict[id_number] = src_transform_copy
                    file_dict[subfolder] = [ build_kdtree_from_npy_file(os.path.join(dir_path,file)), os.path.join(path,file),generate_filename_dict(subfolder),src_crs,transform_dict]



print(file_dict)
print(crs_dict)

{'/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001': [<scipy.spatial._ckdtree.cKDTree object at 0x79f58858fd10>, '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/bounds_array_MODIS_NPP_2001.npy', {0: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID0N48_51114628138105S46_71027383528823W8_981031902268322E10_781904348361152.tif', 1: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID1N48_51114628138105S46_71027383528823W10_777662470507366E12_578534916600194.tif', 2: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID2N48_51114628138105S46_71027383528823W12_574293038746408E14_375165484839236.tif', 3: '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/YearlyValue/MODIS_NPP/2001/ID3N50_3077768496201S48_50690440352727W8_981031902

In [ ]:
path

'/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/SeasonalValue/TotalEvapotranspiration'

In [ ]:
subfolder = Elevation_Path_static
# Get a list of all files in the subfolder
files = os.listdir(subfolder)

                    # Sort the list of files
files.sort()

                    # Get the first file
first_file = files[0]

                    # If you need the full path to the file
first_file_path = os.path.join(subfolder, first_file)
src_dataset = rasterio.open(first_file_path)
                    # Deep copy the source CRS
src_crs = copy.deepcopy(src_dataset.crs)
                    # Close the source dataset
src_dataset.close()
                        # Iterate over the files in the folder
transform_dict = {}
for filename in os.listdir(subfolder):
  file_path = os.path.join(subfolder, filename)
                   # Extract the ID number from the filename
  match = re.search(r'ID(\d+)N', filename)
  if match:
    id_number = int(match.group(1))

                              # Read the rasterio src.transform
  with rasterio.open(file_path) as src:
    src_transform_copy = copy.deepcopy(src.transform)

                              # Store the transform in the dictionary with the ID number as the key
    transform_dict[id_number] = src_transform_copy
file_dict[subfolder] = [ build_kdtree_from_npy_file(os.path.join('/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/StaticValue/','bounds_array_Elevation.npy')), os.path.join('/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/RasterBandsData/StaticValue/','bounds_array_Elevation.npy'),generate_filename_dict(subfolder),src_crs,transform_dict]


In [ ]:

def generate_filename_dict(folder_path):
    filename_dict = {}

    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Extract the ID number from the filename
        match = re.search(r'ID(\d+)N', filename)
        if match:
            id_number = int(match.group(1))

            # Store the filename in the dictionary with the ID number as the key
            filename_dict[id_number] = file_path

    return filename_dict

In [ ]:
def transform_coordinates(lon, lat, crs):
    # Coordinates in the target CRS
    x, y = transform({'init': 'EPSG:4326'}, crs, [lon], [lat])
    return x[0], y[0]

In [ ]:

def get_tif_ID( latitude, longitude,kdTree,DataBoundsArray,DictFilenames,src_crs):
    x, y = transform_coordinates(longitude, latitude,src_crs)
    index_list = find_closest_indices(kdTree,[y,x])
    DataBoundsArray = np.load(DataBoundsArray)
    id_list = DataBoundsArray[index_list, 0]

    for id in id_list:
        filename = DictFilenames[id]
        # Replace underscores with periods
        filenameTransformed = filename.replace('_', '.')
        # Open the raster file using rasterio
        #     # Read the raster data (assuming only one band)
        #                     # Check if all data points have the same value
        #     if (raster_data == raster_data[0]).all():
        #       print('All the same value for ',filename)
        #     else:
              # Extract the four floats using regular expressions
        coordinates = re.findall(r'[NSWE]-?\d+\.\d+|[NSWE]0', filenameTransformed)
              # Convert the strings to floats
        coordinates = [float(coord[1:]) for coord in coordinates]
              # coordinates = [0.0 if coord.endswith('0') else float(coord[1:]) if coord != 'E0' else 0.0 for coord in coordinates]

              # Check if the sampled coordinates match the filename
        if coordinates[3] >= x and coordinates[2]  <= x and coordinates[0] >= y and coordinates[1] <= y:
          return x,y, id
        # else:
        #   return 0,0,0
           # print(coordinates,x,y,id)

    #print('f')
    #print(latitude, longitude)
    return 0,0,0

def get_tif_ArrayPosition( latitude, longitude,kdTree,DataBoundsArray,DictFilenames,src_crs,DictTransform):

  x_original,y_original, id = get_tif_ID( latitude, longitude,kdTree,DataBoundsArray,DictFilenames,src_crs)
  x, y = rasterio.transform.rowcol(DictTransform[id], x_original, y_original)
  if x < 0:
    print(x,y,id)
  if y < 0:
    print(y,x,id)
  return latitude, longitude , id, x , y


In [ ]:
# Specify the path
path = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates'


In [ ]:
df_sample.head(3)

,Longitude,Latitude
0,9.546802,47.866430
1,10.576114,50.424199
2,12.870875,47.267871


In [ ]:
len(file_dict.keys())

503

In [ ]:
# Assume file_dict.keys() contains a list of paths
start_key = 'SeasonalValue/LAI/2022_spring'
done = 0
start_processing = False

In [ ]:
for subfolder in reversed(file_dict.keys()):
  # If the start key is found, enable processing
  # Extract the last 3 parts of the path
  last_three_parts = '/'.join(subfolder.split(os.sep)[-3:])

  if last_three_parts == start_key:
    start_processing = True
    print(done)
    continue  # Skip the start key itself
  # Only process keys after the start key
  if start_processing:
    samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[subfolder][0] , file_dict[subfolder][1] , file_dict[subfolder][2] , file_dict[subfolder][3] , file_dict[subfolder][4]), df_sample.to_numpy())))
    # Get the last three subfolders in the path
    subfolders = subfolder.split(os.sep)[-3:]

    # Create subfolders within the path
    new_path = os.path.join(path, *subfolders)

    # Check if the new path exists, if not, create it
    if not os.path.exists(new_path):
      os.makedirs(new_path)
    print(new_path)
    # Specify the file name
    filename = 'coordinates.npy'

    # Specify the full path to the file
    file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
    np.save(file_path, samplingOn)
  done += 1


295
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2022_winter
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2021_autumn
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2021_summer
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2021_spring
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2021_winter
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2020_autumn
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2020_summer
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2020_spring
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2020_winter
/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/Coordinates/SeasonalValue/LAI/2019_autumn
/content/drive/MyDrive/Col

# Yearly: get OC Coordinates

In [ ]:
import pandas as pd

# Specify the path to the Excel file
filename = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/LUCAS_LFU_Lfl_00to23_Bavaria_OC.xlsx'

# Read the 'GPS_LONG', 'GPS_LAT', and 'year' columns
df = pd.read_excel(filename, usecols=['GPS_LONG', 'GPS_LAT', 'year'])

print(df)


In [ ]:
pathLUCAS = '/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/LUCAS_LFU_Lfl_00to23_Bavaria_OC.xlsx'

In [ ]:
LUCAScoordinates = "/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates"

In [ ]:

for subfolder in file_dict.keys():
    # Get the last subfolder in the path
    last_subfolder = subfolder.split(os.sep)[-1]
    print(subfolder)
    subfolders = subfolder.split(os.sep)[-3:]

    # Try to convert the last subfolder to an integer (assuming the year is represented as an integer)
    try:
        year = int(last_subfolder)
    except ValueError:
        continue  # If it can't be converted to an integer, skip this iteration of the loop

    # Filter the DataFrame based on the year
    df_year = df[df['year'] == year]
    # Remove rows where 'GPS_LONG' or 'GPS_LAT' are not finite numbers
    df_year = df_year[np.isfinite(df_year['GPS_LONG']) & np.isfinite(df_year['GPS_LAT'])]
    # If the DataFrame is empty, skip this iteration of the loop
    if df_year.empty:
        continue
    samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[subfolder][0] , file_dict[subfolder][1] , file_dict[subfolder][2] , file_dict[subfolder][3] , file_dict[subfolder][4]), df_year.to_numpy())))
    # Get the last three subfolders in the path

    # Create subfolders within the path
    new_path = os.path.join(LUCAScoordinates, *subfolders)

    #  Check if the new path exists, if not, create it
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    print(new_path)
    # Specify the file name
    filename = 'coordinates.npy'

    # Specify the full path to the file
    file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
    np.save(file_path, samplingOn)


In [ ]:

    # Get the last subfolder in the path
last_subfolder = Elevation_Path_static.split(os.sep)[-1]
subfolders = Elevation_Path_static.split(os.sep)[-3:]
last_subfolder2 = Elevation_Path_static.split(os.sep)[-2]

    # Try to convert the last subfolder to an integer (assuming the year is represented as an integer)
     # If it can't be converted to an integer, skip this iteration of the loop

    # Filter the DataFrame based on the year
df_year = df
    # Remove rows where 'GPS_LONG' or 'GPS_LAT' are not finite numbers
df_year = df_year[np.isfinite(df_year['GPS_LONG']) & np.isfinite(df_year['GPS_LAT'])]
    # If the DataFrame is empty, skip this iteration of the loop

samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[Elevation_Path_static][0] , file_dict[Elevation_Path_static][1] , file_dict[Elevation_Path_static][2] , file_dict[Elevation_Path_static][3] , file_dict[Elevation_Path_static][4]), df_year.to_numpy())))
    # Get the last three subfolders in the path

    # Create subfolders within the path
new_path = os.path.join(LUCAScoordinates, *subfolders)

    #  Check if the new path exists, if not, create it
if not os.path.exists(new_path):
  os.makedirs(new_path)
print(new_path)
    # Specify the file name
filename = 'coordinates.npy'



In [ ]:
# Specify the full path to the file
file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
np.save(file_path, samplingOn)

In [ ]:
len(samplingOn)

# Season: get OC Coordinates

In [ ]:
pathSeason = "/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/LUCAS_LFU_Bavaria_OC_joint_data_modified.xlsx"

In [ ]:
import pandas as pd

# Specify the path to the Excel file
filename = pathSeason
# Read the 'GPS_LONG', 'GPS_LAT', and 'survey_date' columns
df = pd.read_excel(filename, usecols=['GPS_LONG', 'GPS_LAT', 'survey_date'])

# Convert 'survey_date' to datetime format
df['survey_date'] = pd.to_datetime(df['survey_date'])

# Extract the year and month from 'survey_date'
df['year'] = df['survey_date'].dt.year
df['month'] = df['survey_date'].dt.month

# Define the months for each season
seasons = {
    'winter': [12, 1, 2],
    'spring': [3, 4, 5],
    'summer': [6, 7, 8],
    'autumn': [9, 10, 11]
}

# Create a 'season' column based on the 'month' column
df['season'] = df['month'].apply(lambda x: next((k for k, v in seasons.items() if x in v), None))

# Create a 'year_season' column
df['year_season'] = df['year'].astype(str) + '_' + df['season']

# Create a list of years
years = list(range(2000, 2024))  # This will create a list from 2000 to 2023

# Filter the DataFrame based on the year and season
df_filtered = df[df['year'].isin(years) & df['season'].isin(seasons.keys())]

print(df_filtered)


In [ ]:

for subfolder in file_dict.keys():
    # Get the last subfolder in the path
    last_subfolder = subfolder.split(os.sep)[-1]
    subfolders = subfolder.split(os.sep)[-3:]
    # Try to convert the last subfolder to an integer (assuming the year is represented as an integer)
    try:
        year_season = last_subfolder
    except ValueError:
        continue  # If it can't be converted to an integer, skip this iteration of the loop

    # Filter the DataFrame based on the year
    df_year_season = df[df['year_season'] == year_season]
    # Remove rows where 'GPS_LONG' or 'GPS_LAT' are not finite numbers
    df_year_season = df_year_season[np.isfinite(df_year_season['GPS_LONG']) & np.isfinite(df_year_season['GPS_LAT'])]
    # If the DataFrame is empty, skip this iteration of the loop
    if df_year_season.empty:
        continue
    samplingOn = np.array(list(map(lambda coord: get_tif_ArrayPosition(float(coord[1]), float(coord[0]), file_dict[subfolder][0] , file_dict[subfolder][1] , file_dict[subfolder][2] , file_dict[subfolder][3] , file_dict[subfolder][4]), df_year_season.to_numpy())))
    # Get the last three subfolders in the path

    # Create subfolders within the path
    new_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/MappingSOC/Data/OC_LUCAS_LFU_LfL_Coordinates', *subfolders)

    #  Check if the new path exists, if not, create it
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    print(new_path)
    # Specify the file name
    filename = 'coordinates.npy'

    # Specify the full path to the file
    file_path = os.path.join(new_path, filename)

    # Save the numpy matrix
    np.save(file_path, samplingOn)
